In [1]:
import pytesseract
import numpy as np
import cv2
import tkinter as tk
from tkinter import *
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk
import urllib
import os
from tensorflow.keras.preprocessing import image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'
from pathlib import Path
##https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-setup-3.05.02-20180621.exe
##pip install pytesseract

root = tk.Tk()
root.title("Extract Text From Image")

canvas = tk.Canvas(root, width=600, height=300)
canvas.grid(columnspan=3, rowspan=3)

logo = Image.open('logo.png')
logo = ImageTk.PhotoImage(logo.resize((200, 200)))
logo_label = tk.Label(image=logo)
logo_label.image = logo
logo_label.grid(column=1, row=0)


def textExtract():
    # Browse Image
    browse_text2.set("Text Extraction")
    filename = askopenfile(parent=root, mode='rb', title="Select A file", filetypes=[('image files', '*.png;*.jpg')])
    imagefile = str(filename)[26:-2]
    
    # Display Image
    logo = Image.open(imagefile)
    logo = ImageTk.PhotoImage(logo.resize((200, 200)))
    logo_label = tk.Label(image=logo)
    logo_label.image = logo
    logo_label.grid(column=1, row=0)
    
    # File Path
    path = os.getcwd()
    pathString = str(path)
    pathSplit = pathString.split('\\')
  
    #Select Image Path
    imagePath = Path(imagefile)
    imagePathString = str(imagePath)
    imagePathSplit = imagePathString.split('\\')
    
    # Compare Two Different Path
    originalPath = len(pathSplit)
    predictImgPath = ""
    
    for i in range(originalPath, len(imagePathSplit)):
        if(i == (len(imagePathSplit) - 1)):
            predictImgPath += imagePathSplit[i]
        else:
            predictImgPath += imagePathSplit[i] + "/"
    
    #Predict Image
    textExtract.img = image.load_img(predictImgPath)
    afterPredict = ""
    afterPredict = pytesseract.image_to_string(textExtract.img, config='--psm 6')
    display(np.array(textExtract.img),afterPredict)


def rotate():
    gray_img=cv2.cvtColor(np.array(textExtract.img), cv2.COLOR_BGR2GRAY)
    gray_img=cv2.bitwise_not(gray_img)
    coordinates = np.column_stack(np.where(gray_img > 0))
    angle=cv2.minAreaRect(coordinates)[-1]
    if angle<-45:

    	angle=-(90+angle)

    else:

    	angle=-angle

    height, width = np.array(textExtract.img).shape[:2]
    center_img = (width / 2, height / 2)
    rotationMatrix = cv2.getRotationMatrix2D(center_img, angle, 1.0)

    rotated_img = cv2.warpAffine(np.array(textExtract.img), rotationMatrix, (width, height), borderMode = cv2.BORDER_REFLECT)   
    afterPredict = pytesseract.image_to_string(np.array(rotated_img), config='--psm 6')
    
    display(rotated_img,afterPredict)
    
    
def erosion():
    kernel = np.ones((3,3),np.uint8) #To define the structuring element
    erosion = cv2.erode(np.array(textExtract.img),kernel,iterations = 5)
    
    afterPredict = pytesseract.image_to_string(erosion, config='--psm 6')
    display(erosion,afterPredict)
    
def dilation():
    kernel = np.ones((5,5),np.uint8)
    dilation = cv2.dilate(np.array(textExtract.img),kernel,iterations = 5)
    
    afterPredict = pytesseract.image_to_string(dilation, config='--psm 6')
    display(dilation,afterPredict)
    
def opening():
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(np.array(textExtract.img), cv2.MORPH_OPEN, kernel)
    
    afterPredict = pytesseract.image_to_string(opening, config='--psm 6')
    display(opening,afterPredict)
    
def closing():
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(np.array(textExtract.img), cv2.MORPH_CLOSE, kernel)
    
    afterPredict = pytesseract.image_to_string(closing, config='--psm 6')
    display(closing,afterPredict)

def averageFiltering():
    gray_imgNoise = cv2.cvtColor(np.array(textExtract.img), cv2.COLOR_BGR2GRAY)
    blurAverage = cv2.blur(gray_imgNoise,(5,5))
    
    afterPredict = pytesseract.image_to_string(blurAverage, config='--psm 6')
    display(blurAverage,afterPredict)
    
def gaussianFiltering():
    gray_imgNoise = cv2.cvtColor(np.array(textExtract.img), cv2.COLOR_BGR2GRAY)
    blurGauss = cv2.GaussianBlur(gray_imgNoise,(5,5),0)
    
    afterPredict = pytesseract.image_to_string(blurGauss, config='--psm 6')
    display(blurGauss,afterPredict)
    
def medianFiltering():
    gray_imgNoise = cv2.cvtColor(np.array(textExtract.img), cv2.COLOR_BGR2GRAY)
    blurMedian = cv2.medianBlur(gray_imgNoise,5)
    
    afterPredict = pytesseract.image_to_string(blurMedian, config='--psm 6')
    display(blurMedian,afterPredict)
    
def original():
    afterPredict = pytesseract.image_to_string(np.array(textExtract.img), config='--psm 6')
    display(np.array(textExtract.img),afterPredict)
    
def display(img,afterPredict):
    logo = Image.fromarray(img)
    logo = ImageTk.PhotoImage(logo.resize((200, 200)))
    logo_label = tk.Label(image=logo)
    logo_label.image = logo
    logo_label.grid(column=1, row=0)
    
    text_box = tk.Text(root, height=10, width=50, padx=15, pady=15)
    text_box.insert(1.0, afterPredict)
    text_box.tag_configure("center", justify="center")
    text_box.tag_add("center", 1.0, "end")
    text_box.grid(column=1, row=3)    
    
#add button
btnErosion = Button(root, text='Erotion', width=7,
             height=1, bd='10', command=erosion)  
btnErosion.place(x=420, y=0)

btnDilation = Button(root, text='Dilation', width=7,
             height=1, bd='10', command=dilation)  
btnDilation.place(x=420, y=40)

btnOpening = Button(root, text='Opening', width=7,
             height=1, bd='10', command=opening)  
btnOpening.place(x=420, y=80)

btnClosing = Button(root, text='Closing', width=7,
             height=1, bd='10', command=closing)  
btnClosing.place(x=420, y=120)

btnRotate = Button(root, text='Rotate', width=7,
             height=1, bd='10', command=rotate)
btnRotate.place(x=420, y=160)

btnAvgFilter = Button(root, text='Average', width=7,
             height=1, bd='10', command=averageFiltering)
btnAvgFilter.place(x=520, y=0)

btnGauFilter = Button(root, text='Gaussian', width=7,
             height=1, bd='10', command=gaussianFiltering)
btnGauFilter.place(x=520, y=40)

btnMedFilter = Button(root, text='Median', width=7,
             height=1, bd='10', command=medianFiltering)
btnMedFilter.place(x=520, y=80)

btnOri = Button(root, text='Original', width=7,
             height=1, bd='10', command=original)
btnOri.place(x=520, y=120)

    

browse_text2 = tk.StringVar()
browse_btn2 = tk.Button(root, textvariable=browse_text2, command=lambda:textExtract(), font="Raleway", bg="#20bebe", fg="white", height=1, width=15)
browse_text2.set("Extract Text")
browse_btn2.grid(column=1, row=2)

canvas = tk.Canvas(root, width=600, height=250)
canvas.grid(columnspan=3)


root.mainloop()